In [38]:
from transformers import pipeline
import os
os.umask(0)


0

In [39]:
short_description = 'a red apple'

input_text = f"short: please describe what you might see in a picture of a scene that contains 'a Christmas tree', write each sentence in a list, and use complete sentences with all nouns and objects you are referring to\n \
                  long: 	1	In the center of the room, a majestic evergreen Christmas tree stands tall, adorned with twinkling lights and colorful ornaments.\n \
            2	Delicate strands of tinsel gracefully drape the tree's branches, adding a touch of shimmer to the festive display.\n \
            3	An elegant star or angel graces the top of the tree, representing the Star of Bethlehem or the heavenly messengers present at Jesus' birth.\n \
            4	Wrapped presents in various shapes and sizes are piled beneath the tree, their festive gift wrap and bows hinting at the surprises inside.\n \
            5	A cozy fireplace crackles nearby, with stockings hung from the mantel, eagerly awaiting the arrival of Santa Claus.\n \
            6	Lush green garlands and flickering candles decorate the mantel, enhancing the holiday atmosphere.\n \
            7	Comfortable seating arrangements, such as sofas and armchairs, are positioned near the tree, complete with plush cushions and warm throw blankets.\n \
            8	Family members and friends gather around the tree in festive attire, sharing laughter and conversation.\n \
            9	A beautifully crafted wreath hangs on a nearby wall or window, adding an additional touch of holiday cheer.\n \
            10	Through the window, a snowy winter landscape can be seen, with snow-covered trees, rooftops, and gently falling snowflakes, creating the perfect backdrop for the Christmas scene.\n \
                  short: please describe what you might see in a picture of a scene that contains 'a male hand playing nervously with a pencil on a black background', write each sentence in a list, and use complete sentences with all nouns and objects you are referring to \n \
                  long:   1	A male hand is positioned prominently in the frame, with fingers flexing and shifting as they manipulate a pencil.\n \
            2	The pencil, held between the thumb and index finger, twirls and spins as the hand moves it nervously.\n \
            3	Shadows from the hand and pencil cast dramatic patterns on the stark black background, emphasizing the sense of tension and unease.\n \
            4	Flecks of graphite from the pencil's tip may be visible, scattered across the black surface, as a result of the restless movements.\n \
            5	The hand's knuckles and veins are accentuated by the lighting, highlighting the pressure and force exerted during the fidgeting.\n \
            6	The pencil's eraser end, worn and discolored, suggests frequent use and a history of anxious behavior.\n \
            7	A hint of perspiration on the hand's skin glistens under the light, further revealing the nervous energy within the scene.\n \
            8	The positioning of the hand, perhaps slightly off-center or at an angle, contributes to the visual tension of the composition.\n \
            9	Fingernails on the hand may appear bitten or worn, indicating a habit of nervousness and stress.\n \
            10	The black background contrasts sharply with the hand and pencil, isolating them in the scene and focusing the viewer's attention on the restless, uneasy motion.\n \
                  short: please describe what you might see in a picture of a scene that contains 'a man is programming', write each sentence in a list, and use complete sentences with all nouns and objects you are referring to \n \
                  long:   1	A focused man sits at a desk, his eyes intently scanning the computer screen in front of him as he works on a programming project.\n \
            2	The computer display is filled with lines of code, featuring various colors and syntax highlighting to differentiate between elements of the programming language.\n \
            3	The man's fingers move swiftly and confidently across the keyboard, typing commands and adjusting the code as needed.\n \
            4	Beside the keyboard, a mouse and a notepad with handwritten notes or algorithms offer additional tools for the programmer's work.\n \
            5	A cup of coffee or tea sits nearby, providing the man with a source of caffeine to maintain his focus and energy.\n \
            6	The room's lighting, either from a desk lamp or overhead lights, illuminates the workspace, creating a comfortable environment for concentration.\n \
            7	The man wears casual attire, such as a t-shirt and jeans, reflecting the informal nature of the programming process.\n \
            8	Reference books or technical manuals may be stacked or spread out on the desk, offering guidance and information for the programmer.\n \
            9	The man's facial expression, furrowed brows or a slight frown, conveys his deep concentration and determination to solve the coding challenge at hand.\n \
            10	Surrounding the man, other electronic devices, like a smartphone or tablet, may be present, indicating the interconnected nature of his work in the digital realm.\n \
                  short: please describe what you might see in a picture of a scene that contains '{short_description}', write each sentence in a list, and use complete sentences with all nouns and objects you are referring to \n \
                  long: "



In [40]:
# original generator
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-2.7B', device=0)

res = generator(input_text, do_sample=True, max_length=len(input_text.split()) + 800,
                          min_length=len(input_text.split()) + 600)
long_description_out = res[0]['generated_text'].replace(input_text, "")
print(long_description_out)
print(generator.model.config)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


  1	A red apple sits on a white plate in a festive Christmas scene shot from above.
             2	The apple's shape is reminiscent of the classic red apple, shaped and dusted by the camera lens.
             3	The apple's red color glistens in the bright light, emphasized by the white plate and clear dishes
GPTNeoConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "EleutherAI/gpt-neo-2.7B",
  "activation_function": "gelu_new",
  "architectures": [
    "GPTNeoForCausalLM"
  ],
  "attention_dropout": 0,
  "attention_layers": [
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local"
  ],
  "attention_typ

In [41]:
# generator with hallucination-reducing parameters
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
model_name = "EleutherAI/gpt-neo-2.7B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Input prompt
prompt = input_text
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Generate text with hallucination-reducing parameters
output = model.generate(
    input_ids,
    max_new_tokens=200,  # Generate up to 200 new tokens
    min_new_tokens=50,  # Generate at least 50 new tokens
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.7, # Controls the randomness of token sampling.
    top_k=50, # Limits sampling to the top-k tokens with the highest probabilities.
    top_p=0.9, # Includes only the smallest set of tokens whose cumulative probability exceeds p.
    repetition_penalty=1.2, # Penalize repeated phrases
    no_repeat_ngram_size=3, # Prevent repeated bigrams
)

# # Decode and print the result

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
# Remove the input prompt from the output if necessary
if generated_text.startswith(input_text):
    generated_text = generated_text[len(input_text):].strip()

print("Raw Generated Text:\n", generated_text)

/home/xzhon54/miniconda3/envs/new_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/xzhon54/miniconda3/envs/new_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Raw Generated Text:
 1   A red apple sits on a table, surrounded by a variety of other fruits and vegetables.
2   In the center, a red apple is placed on a plate, surrounded with other fruits, vegetables, and other foods.
3   The apple is surrounded by other fruits.
4   Other fruits surround the apple.
5   Fruits surround the red apple.


In [42]:
from sentence_transformers import SentenceTransformer, util

similarity_model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight semantic similarity model

def filter_hallucinations(input_text, expanded_text, min_similarity=0.7):
    """
    Filter hallucinated sentences by checking semantic similarity with the input context.
    """
    input_embedding = similarity_model.encode(input_text, convert_to_tensor=True)
    sentences = expanded_text.split("\n")
    filtered_sentences = []

    for sentence in sentences:
        if sentence.strip():  # Ignore empty sentences
            sentence_embedding = similarity_model.encode(sentence, convert_to_tensor=True)
            similarity = util.pytorch_cos_sim(input_embedding, sentence_embedding).item()
            if similarity >= min_similarity:
                filtered_sentences.append(sentence)

    return filtered_sentences

In [43]:
print("Raw Generated Text:\n", generated_text)
filtered_sentences = filter_hallucinations(short_description, generated_text, 0.7)
print("\nFiltered Sentences (Reduced Hallucination):\n", filtered_sentences)

Raw Generated Text:
 1   A red apple sits on a table, surrounded by a variety of other fruits and vegetables.
2   In the center, a red apple is placed on a plate, surrounded with other fruits, vegetables, and other foods.
3   The apple is surrounded by other fruits.
4   Other fruits surround the apple.
5   Fruits surround the red apple.

Filtered Sentences (Reduced Hallucination):
 ['1   A red apple sits on a table, surrounded by a variety of other fruits and vegetables.', '5   Fruits surround the red apple.']
